In [1]:
import pandas as pd
import pickle

In [6]:
#1. 두 데이터 셋 라벨 합침 
import csv

rows1 = []
last_number=0
# Load dataset1
with open('/home/irteam/junghye-dcloud-dir/Pixt/data/dataset/dataset1/label.csv', 'r') as f:
    csv_data = csv.reader(f)
    header=next(csv_data)
    rows1 = list(csv_data)
   

# Find the last number in dataset1
last_number = int(rows1[-1][0])
print(last_number)

# Load dataset2 and adjust the numbers
with open('/home/irteam/junghye-dcloud-dir/Pixt/data/dataset/dataset2/label.csv', 'r') as f:
    csv_data = csv.reader(f)
    header=next(csv_data)
    for row in csv_data:
        # Increment the number and append the row to dataset1
        last_number += 1
        row[0] = str(last_number)
        
        rows1.append(row)

# Print the combined dataset
print('변경 후',len(rows1))

with open('/home/irteam/junghye-dcloud-dir/Pixt/MotionExcluded/dataset/OriginalMerged_data.csv','w') as outfile:
    outfile.truncate()
    wr=csv.writer(outfile)
    wr.writerows(rows1)

3963
변경 후 5963


희소 라벨 제거
- 빈도 수 구하고, 1,2인 것들 태그는 모두 제거
- 만약 태그 제거했더니 태그가 없으면 데이터도 삭제


In [19]:
# 희소 라벨 제거 
# 희소 라벨 제거 했더니 태그 없으면 데이터 자체도 제거 
import csv
from collections import Counter
import os

# Load dataset
input_file = '/home/irteam/junghye-dcloud-dir/Pixt/MotionExcluded/dataset/OriginalMerged_data.csv'
output_file = '/home/irteam/junghye-dcloud-dir/Pixt/MotionExcluded/dataset/OriginalMerged_data_filtered.csv'
list_file='/home/irteam/junghye-dcloud-dir/Pixt/MotionExcluded/dataset/DeleteList.txt'
removePath1='/home/irteam/junghye-dcloud-dir/Pixt/MotionExcluded/dataset/dataset1'
removePath2='/home/irteam/junghye-dcloud-dir/Pixt/MotionExcluded/dataset/dataset2'


rows = []
all_labels = []

# Load data and collect all labels
with open(input_file, 'r') as f:
    csv_data = csv.reader(f)
    rows = list(csv_data)
    all_labels = [tag for row in rows for tag in row[2:]]

print('태그 제거 전(전체 태그):', len(all_labels))
print('태그 제거 전(unique 태그)',len(set(all_labels)))
# Count the frequency of each label
counter = Counter(all_labels)

# Find tags with frequency <= 2
to_delete_tags = [key for key, item in counter.items() if item <= 2]


print('지울 태그 개수:', len(to_delete_tags))
# to_delete_tags list saved
with open(list_file,'w') as f:
    f.truncate()
    f.write('\n'.join(to_delete_tags))

# Remove tags with frequency <= 2 from each row
filtered_rows = []
dataset1_Length=3963

for row in rows:
    serial_title=row[:2]
    filtered_labels = [value for value in row[2:] if value not in to_delete_tags]
    if filtered_labels:
        filtered_row=serial_title+filtered_labels
        filtered_rows.append(filtered_row)
    else:
        # delete data
        serial_number=int(serial_title[0])
        if serial_number<=dataset1_Length:
            os.remove(os.path.join(removePath1,str(serial_number)+'.webp'))
        else:
            serial_number=serial_number-dataset1_Length
            os.remove(os.path.join(removePath2,str(serial_number)+'.webp'))
            
            
        

# Save the filtered data to a new CSV file
with open(output_file, 'w', newline='') as f:
    f.truncate()
    csv_writer = csv.writer(f)
    csv_writer.writerows(filtered_rows)

print(f'희소 라벨 제거 + 일부 데이터 제거 데이터가 성공적으로 {output_file}에 저장되었습니다.')


   

태그 제거 전(전체 태그): 74203
태그 제거 전(unique 태그) 7578
지울 태그 개수: 4355
희소 라벨 제거 + 일부 데이터 제거 데이터가 성공적으로 /home/irteam/junghye-dcloud-dir/Pixt/MotionExcluded/dataset/OriginalMerged_data_filtered.csv에 저장되었습니다.


In [61]:
import glob 
dt1=glob.glob ('/home/irteam/junghye-dcloud-dir/Pixt/MotionExcluded/dataset/dataset1/*.webp')
dt2=glob.glob('/home/irteam/junghye-dcloud-dir/Pixt/MotionExcluded/dataset/dataset2/*.webp')

print(len(dt1),len(dt2))

3963 2000


제거된 데이터는 없음을 확인 

In [133]:
# 새 annotation file 작성


from collections import defaultdict


filtered_data='/home/irteam/junghye-dcloud-dir/Pixt/MotionExcluded/dataset/OriginalMerged_data_filtered.csv'
with open(output_file,'r')as f:
    data=csv.reader(f)
    rows=list(data)

title=[row[1] for row in rows]
serial_num=[row[0] for row in rows]
tags=[row[2:] for row in rows]
dataset = ['dataset1' if int(num) <= 3963 else 'dataset2' for num in [row[0] for row in rows]]
motion_exists=[1  if '모션' in row[2:] else 0 for row in rows]
greenScreen_exists=[1 if '그린스크린' in row[2:] else 0 for row in rows]

# 중복된 title 수정
title_dict = defaultdict(int)  
modified_title = []  # newtitle list

for title_value in title:
    if title_dict[title_value] == 0:
        modified_title.append(title_value)  
    else:
        modified_title.append(f"{title_value}_{title_dict[title_value]}")  # if duplicated, add number
    title_dict[title_value] += 1  

title = modified_title 

tag_items=dict(zip(title,tags))
serial_num_items=dict(zip(title,serial_num))
motion_items=dict(zip(title,motion_exists))
greenScreen_items=dict(zip(title,greenScreen_exists))
dataset_items=dict(zip(title,dataset))

labels=defaultdict(Tags=tag_items,Serial_num=serial_num_items,Motion_exists=motion_items,GreenScreen_exists=greenScreen_items,Dataset=dataset_items)

print(labels.keys())

dict_keys(['Tags', 'Serial_num', 'Motion_exists', 'GreenScreen_exists', 'Dataset'])


In [134]:
print(len(tag_items),len(serial_num_items))


5963 5963


In [135]:
# pickle 파일로 저장 
import pickle
label_temp=pd.DataFrame(labels)
label_temp.to_pickle('/home/irteam/junghye-dcloud-dir/Pixt/MotionExcluded/dataset/label_0528.pkl')

In [136]:
label_0528=pd.read_pickle('/home/irteam/junghye-dcloud-dir/Pixt/MotionExcluded/dataset/label_0528.pkl')

label_0528.reset_index(inplace=True)
label_0528 = label_0528.rename(columns={'index': 'title'})
label_0528

,title,Tags,Serial_num,Motion_exists,GreenScreen_exists,Dataset
0,세개의 크리스마스 트리,"[조명, 나무, 모션, 그린스크린, 겨울, 크리스마스, 산타, 캐롤, 모션그래픽, ...",1,1,1,dataset1
1,저수지마을_구름_일몰,"[시골, 일몰, 마을, 구름, 지붕, 전경, 산, 강물, 강, 타임랩스, 강가, 저...",2,0,0,dataset1
2,염전 곰소 드론 -20 _2160p_30fps_prores422,"[바다, 농사, 드론, 갯벌, 염전, 곰소, 소금, 바닷물, 천일염, 서해안, 신안]",3,0,0,dataset1
3,변산 솔섬 해변 일몰 석양 노을 -2 _2160p_60fps_prores422,"[나무, 일몰, 자연, 바다, 해, 산, 노을, 바닷가, 소나무, 석양, 변산, 전...",4,0,0,dataset1
4,20210724_제스처_권미정_손가락을 비비다,"[인물, 그린스크린, 손, 손가락, 제스쳐, 비비다, 크로마키]",5,0,1,dataset1
...,...,...,...,...,...,...
5958,덕유산겨울풍경24,"[겨울, 눈, 등산, 오리, 케이블카, 눈꽃, 겨울등산, 상고대, 눈보라, 영하, ...",5959,0,0,dataset2
5959,할로윈_무덤_배경소스,"[모션, 손, 배경, 달, 백그라운드, 전설, 공포, 땅, 달빛, 무서운, 할로윈,...",5960,1,0,dataset2
5960,봄11,"[꽃, 모션, 풍경, 자연, 사랑, 배경, 연인, 웨딩, 낭만, 꽃밭, 결혼, 커플...",5961,1,0,dataset2
5961,눈 위의 소나무 가지 (원경),"[자연, 겨울, 눈, 12월, 햇살, 추운, snow, 나무가지, nature, w...",5962,0,0,dataset2


In [137]:
# csv 파일로 저장 
label_0528.to_csv('/home/irteam/junghye-dcloud-dir/Pixt/MotionExcluded/dataset/label_0528.csv',index=False)

In [138]:
# csv 파일 확인 
label_0528_csv=pd.read_csv('/home/irteam/junghye-dcloud-dir/Pixt/MotionExcluded/dataset/label_0528.csv')
label_0528_csv

,title,Tags,Serial_num,Motion_exists,GreenScreen_exists,Dataset
0,세개의 크리스마스 트리,"['조명', '나무', '모션', '그린스크린', '겨울', '크리스마스', '산타...",1,1,1,dataset1
1,저수지마을_구름_일몰,"['시골', '일몰', '마을', '구름', '지붕', '전경', '산', '강물'...",2,0,0,dataset1
2,염전 곰소 드론 -20 _2160p_30fps_prores422,"['바다', '농사', '드론', '갯벌', '염전', '곰소', '소금', '바닷...",3,0,0,dataset1
3,변산 솔섬 해변 일몰 석양 노을 -2 _2160p_60fps_prores422,"['나무', '일몰', '자연', '바다', '해', '산', '노을', '바닷가'...",4,0,0,dataset1
4,20210724_제스처_권미정_손가락을 비비다,"['인물', '그린스크린', '손', '손가락', '제스쳐', '비비다', '크로마키']",5,0,1,dataset1
...,...,...,...,...,...,...
5958,덕유산겨울풍경24,"['겨울', '눈', '등산', '오리', '케이블카', '눈꽃', '겨울등산', ...",5959,0,0,dataset2
5959,할로윈_무덤_배경소스,"['모션', '손', '배경', '달', '백그라운드', '전설', '공포', '땅...",5960,1,0,dataset2
5960,봄11,"['꽃', '모션', '풍경', '자연', '사랑', '배경', '연인', '웨딩'...",5961,1,0,dataset2
5961,눈 위의 소나무 가지 (원경),"['자연', '겨울', '눈', '12월', '햇살', '추운', 'snow', '...",5962,0,0,dataset2
